# Automated UAS SkyWatch Standard Format Remark Functions & Script

## Origin Location: RWY
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** October 29, 2021

***
***How to Use:***

`This function will output the dataframe containing the complete UAS location information for Standard Format Remarks referencing a Runway (RWY).`
***

## Import Statements

In [2]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

# REMARK MUST REFERENCE A RUNWAY

### uas_rwy() 

This function determines the records for which the Remarks reference a Runway.

In [94]:
def uas_rwy(file):
    '''
    This function takes in a file (str) containing unidentified Remarks from SkyWatch and determines those that reference a 
    runway as the origin location for the UAS sighting. This function outputs a dataframe containing the Remark, the Runway 
    Location, the Reporting Facility (needed to find the airport in relation to the runway referenced), and the UAS Distance
    from the runway (if provided)
    
    @param file: a .csv file containing unidentified Remarks from SkyWaych
    '''
    
    # Read in .csv file provided by user
    unid_remarks = pd.read_csv(file)
    
    # Create List that contains each Standard Remark, and the Heading/Direction
    # information contained in each remark

    remark_uas_loc = []
    remarks = unid_remarks['REMARKS']

    # regular expression for any heading/direction
    rwy_regex = 'RWY\s?[0-9]*[A-Z]?'

    # Loop through all remarks and search for the heading/direction regex above
    for i in range(len(remarks)):
        rwy = re.findall(rwy_regex, remarks[i])
        remark_uas_loc.append(remarks[i])
        remark_uas_loc.append(rwy)

    # Split Remarks and Heading/Directions into two seperate lists and create
    # pandas dataframe
    remark = []
    uas_loc = []

    for i in range(0, len(remark_uas_loc), 2):
        remark.append(remark_uas_loc[i])
        uas_loc.append(remark_uas_loc[i+1])

    remark_uas_loc_df = pd.DataFrame()
    remark_uas_loc_df['REMARKS'] = remark
    remark_uas_loc_df['UAS_Location_Runways'] = uas_loc
    
    # Get list of UAS locations from the above dataframe
    uas_loc = remark_uas_loc_df['UAS_Location_Runways'] 

    # If the regular expressions did not hit on any location information, pass it UNKN for now
    for i in range(len(remark_uas_loc_df)):
        if len(uas_loc[i]) == 0:
            uas_loc[i] = 'UNKN'
    
    # NOTE :: remark_uas_loc_df is the dataframe that contains ALL records
    #         rwy_df is the dataframe that contains ONLY RECORDS THAT REFERENCES RWY
    
    rwy_df = remark_uas_loc_df[remark_uas_loc_df['UAS_Location_Runways'] != 'UNKN']
    rwy_df = rwy_df.reset_index()
    
    # Read in Output_All_Points.csv from Lex
    output_all_points = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Output_All_Points.csv',encoding='cp1252')
    
    rwy_complete_df = pd.merge(rwy_df, output_all_points, on='REMARKS', how='left')
    rwy_complete_df = rwy_complete_df.drop_duplicates(subset = ['REMARKS'])
    std_rwyComplete = rwy_complete_df[['REMARKS','UAS_Location_Runways', 'UASLOCATION','REPORTINGFACILITY','RWYLOCATION']].reset_index()
    
    # Rename the 'UASLOCATION' field to 'IDENT' for merging purposes
    std_rwyComplete.rename(columns = {'REPORTINGFACILITY': 'IDENT'}, inplace = True)

    # Create List that contains each Standard Remark, and the Heading/Direction
    # information contained in each remark

    remark_uas_loc = []
    remarks = std_rwyComplete['REMARKS']

    # regular expression for any heading/direction
    headir_regex = '\.?[0-9]\.?[0-9]*[0-9]*\s?NM'

    # Loop through all remarks and search for the heading/direction regex above
    for i in range(len(remarks)):
        head_dir = re.findall(headir_regex, remarks[i])
        remark_uas_loc.append(remarks[i])
        remark_uas_loc.append(head_dir)

    # Split Remarks and Heading/Directions into two seperate lists and create
    # pandas dataframe
    remark = []
    uas_loc = []

    for i in range(0, len(remark_uas_loc), 2):
        remark.append(remark_uas_loc[i])
        uas_loc.append(remark_uas_loc[i+1])

    # Append UAS Distance
    std_rwyComplete['UAS_Distance'] = uas_loc
    
    # Export final DF to .csv 
    std_rwyComplete.to_csv('runways%d.csv'%len(std_rwyComplete), index = False)
    
    return std_rwyComplete

In [95]:
df = uas_rwy('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/unidentified_std_remarks.csv')
df

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,0,Aircraft observed a UAS 100 feet above the Air...,[RWY30],NaN,NaN,NaN,[2 NM]
1,1,Aircraft observed an UAS at 200 feet above the...,[RWY8],NaN,NaN,NaN,[]
2,2,Aircraft observed a UAS off his right side at ...,[RWY23],NaN,NaN,NaN,[]
3,3,"Aircraft observed a red and white UAS at 1,000...","[RWY18L, RWY18C]",NaN,NaN,NaN,[]
4,4,"Aircraft observed a black UAS at 1,000 feet wh...",[RWY 32R],PAM,PAM,RWY 32R,[3 NM]
...,...,...,...,...,...,...,...
594,596,Aircraft reported a fixed wing UAS pass within...,[RWY 10],SAV,SAV,RWY 10,[1 NM]
595,597,Aircraft reported a grey quad-copter UAS off t...,[RWY 27L],SFB,SFB,RWY 27L,[]
596,598,Aircraft reported a black quad-copter UAS off ...,[RWY 3R],NaN,NaN,NaN,[]
597,599,Aircraft reported a NMAC with a UAS while on a...,[RWY05],NaN,NaN,NaN,[6 NM]


In [55]:
def drop_nan(dataframe):
    '''
    This function replaces all instances of [] in the dataframe provided as
    input with NaN and then returns a new dataframe with no 'NaN' values
    
    input : dataframe is a dataframe containing UAS remark information
    output : a new dataframe with no 'NaN' values
    '''
    
    df1 = dataframe.mask(dataframe.applymap(str).eq('[]'))
    output_df = df1.dropna().reset_index(drop=True)
    
    return output_df

In [11]:
df1 = drop_nan(df)
#df1

In [56]:
def airRwy_link (dataframe):
    '''
    This function does the following:
        1. Merge the dataframe with the airports_runways_linked.csv dataset
            a. Merge in the IDENT field
        2. Extracts the Runway Designator from RWYLOCATION field
        3. Checks if the Runway Designator found in the Remark matches one
           of the RWY Designators from airports_runways_linked.csv
            a. Only keeps those records for which there is a match
        4. Merge the resulting dataframe with airports_cleaned_declination.csv
           to get the declination for the runway
    
    input : dataframe is a dataframe containing UAS remark information and 
            Runway Location information in a field named RWYLOCATION
    output : a new dataframe in which all records have a runway location
             that is present at an airport in airports_runways_linked.csv
    '''
    # Import and store datasets to merge with
    air_rwy = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_runways_linked.csv')
    air_dec = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned_declination.csv')
    
    # Merge the two datasets on the 'IDENT' field
    rwy_linked = pd.merge(dataframe, air_rwy, on = 'IDENT', how = 'left')
    rwy_linked.rename(columns = {'properties.DESIGNATOR': 'DESIGNATOR'}, inplace = True)
    
    # Extract Runway Designator from RWYLOCATION field
    rwydesignator = []

    for i in range(len(rwy_linked)):
        rwy_split = rwy_linked['RWYLOCATION'][i].split(' ')
        rwydesignator.append(rwy_split[1])

    # Add split column with RWY designators to dataframe as new field
    rwy_linked['RWYDESIGNATOR'] = rwydesignator
    
    # Check if Runway Designator found in Remark matches one of the Runway designators
    # from the air_rwy dataframe
    rwy_linked['RWYinREMARK'] = rwy_linked.apply(lambda x: str(x.RWYDESIGNATOR) in str(x.DESIGNATOR), axis=1)

    # Only keep records for which the RWY Designator is present in the DESIGNATOR field 
    output_df1 = rwy_linked[rwy_linked['RWYinREMARK'] == True]
    
    # Merge resulting dataframe with airport declination (air_dec) dataframe
    output_dec = pd.merge(output_df1, air_dec, on = 'IDENT', how = 'left')
    
    # Only keep necessary columns/fields of interest
    output_df = pd.DataFrame(output_dec[['REMARKS', 'UASLOCATION', 'IDENT', 'RWYLOCATION', 'UAS_Distance', 'RWYDESIGNATOR', 'Runway.Latitude', 'Runway.Longitude', 'DECLINATION']])
    
    return output_df

In [17]:
df2 = airRwy_link(df1)
df2

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway.Latitude,Runway.Longitude,DECLINATION
0,"Aircraft observed a black UAS at 1,000 feet wh...",PAM,PAM,RWY 32R,[3 NM],32R,-85.570825,30.068523,-4.13012
1,Aircraft observed a UAS above while W bound at...,DFW,DFW,RWY 18R,[5 NM],18R,-97.054815,32.893723,3.06900
2,Aircraft observed a UAS at the same altitude w...,MEM,MEM,RWY 36L,[6 NM],36L,-89.987278,35.039251,-1.62738
3,Aircraft observed a red UAS off the right side...,SEA,SEA,RWY 16R,[4 NM],16R,-122.317932,47.449860,15.40471
4,Aircraft observed a small red UAS E bound off ...,ADS,ADS,RWY 15,[3 NM],15,-96.837304,32.970408,2.94849
...,...,...,...,...,...,...,...,...,...
90,Aircraft reported a UAS passing 100 feet below...,TLH,TLH,RWY 27,[2 NM],27,-84.346397,30.391355,-4.96630
91,"Aircraft reported a UAS off the right side, ap...",LAX,LAX,RWY 25L,[3 NM],25L,-118.404501,33.935085,11.71212
92,"Aircraft reported a grey, basketball size, UAS...",LAX,LAX,RWY 25L,[10 NM],25L,-118.404501,33.935085,11.71212
93,Aircraft reported a UAS off the left side whil...,RDU,RDU,RWY 23R,[3 NM],23R,-78.792358,35.882177,-9.22834


In [91]:
def rwyUAS_latLong (dataframe):
    '''
    This function takes in a dataframe containing Runway latitude/longitude
    information as well as runway declination information for each Remark
    having a runway origin reference location.
    
    This function does the following:
        1. Calculates bearing information for each runway reference
        2. Converts distances from NM to kilometers
        3. Calculates UAS Latitude/Longitude information using geopy
    
    input : dataframe containing runway lat/long and declination information
    output : dataframe containing UAS Latitude/Longitude information
    '''
    # Calculate bearing information for each runway reference
    designator = dataframe['RWYDESIGNATOR']
    designator_bearing = []

    for i in range(len(dataframe)):
        designator_bearing.append(int(re.sub("\D","",designator[i]))*10 + dataframe['DECLINATION'][i])

    dataframe['RWY_BEARING'] = designator_bearing
    
    # Convert Distances from NM to kilometers
    dist_kilo = []

    for i in range(len(dataframe)):
        distanceKilo = int(re.sub("\D","",str(dataframe['UAS_Distance'][i])))* 1.852 # converting NM to kilometers
        dist_kilo.append(distanceKilo)

    dataframe['Distance_Kilometers'] = dist_kilo
    
    # BECAUSE ERIC HAD THESE SWITCHED ****************
    dataframe.rename(columns = {'Runway.Latitude': 'Runway_Longitude', 'Runway.Longitude' : 'Runway_Latitude'}, inplace = True)
    
    # Calculate UAS Lat/Long information using geopy
    uas_lat = []
    uas_long = []


    for i in range(len(dataframe)):
        lat_rwy = pd.to_numeric(dataframe['Runway_Latitude'][i])
        long_rwy = pd.to_numeric(dataframe['Runway_Longitude'][i])
        b = pd.to_numeric(dataframe['RWY_BEARING'][i])
        d = pd.to_numeric(dataframe['Distance_Kilometers'][i])

        origin = geopy.Point(lat_rwy, long_rwy)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2, = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)

    # Append UAS Lat/Long information to DataFrame
    dataframe['UAS_Latitude'] = uas_lat
    dataframe['UAS_Longitude'] = uas_long
    
    # Export dataframe containing UAS lat/long information to .csv file 
    dataframe.to_csv('rwy%d_uasLatLong.csv'%len(dataframe), index=False)
    
    return dataframe
    

In [88]:
df3 = rwyUAS_latLong(df2)
df3

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway_Longitude,Runway_Latitude,DECLINATION,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude
0,"Aircraft observed a black UAS at 1,000 feet wh...",PAM,PAM,RWY 32R,[3 NM],32R,-85.570825,30.068523,-4.13012,315.86988,5.556,30.104491,-85.610962
1,Aircraft observed a UAS above while W bound at...,DFW,DFW,RWY 18R,[5 NM],18R,-97.054815,32.893723,3.06900,183.06900,9.260,32.810345,-97.060108
2,Aircraft observed a UAS at the same altitude w...,MEM,MEM,RWY 36L,[6 NM],36L,-89.987278,35.039251,-1.62738,358.37262,11.112,35.139371,-89.990741
3,Aircraft observed a red UAS off the right side...,SEA,SEA,RWY 16R,[4 NM],16R,-122.317932,47.449860,15.40471,175.40471,7.408,47.383443,-122.310072
4,Aircraft observed a small red UAS E bound off ...,ADS,ADS,RWY 15,[3 NM],15,-96.837304,32.970408,2.94849,152.94849,5.556,32.925788,-96.810288
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Aircraft reported a UAS passing 100 feet below...,TLH,TLH,RWY 27,[2 NM],27,-84.346397,30.391355,-4.96630,265.03370,3.704,30.388457,-84.384793
91,"Aircraft reported a UAS off the right side, ap...",LAX,LAX,RWY 25L,[3 NM],25L,-118.404501,33.935085,11.71212,261.71212,5.556,33.927851,-118.463962
92,"Aircraft reported a grey, basketball size, UAS...",LAX,LAX,RWY 25L,[10 NM],25L,-118.404501,33.935085,11.71212,261.71212,18.520,33.910858,-118.602666
93,Aircraft reported a UAS off the left side whil...,RDU,RDU,RWY 23R,[3 NM],23R,-78.792358,35.882177,-9.22834,220.77166,5.556,35.844248,-78.832521


In [92]:
def main (dataframe):
    df = uas_rwy(dataframe)
    df1 = drop_nan(df)
    df2 = airRwy_link(df1)
    output = rwyUAS_latLong(df2)
    
    return output

In [93]:
main('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/unidentified_std_remarks.csv')

,REMARKS,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,RWYDESIGNATOR,Runway_Longitude,Runway_Latitude,DECLINATION,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude
0,"Aircraft observed a black UAS at 1,000 feet wh...",PAM,PAM,RWY 32R,[3 NM],32R,-85.570825,30.068523,-4.13012,315.86988,5.556,30.104491,-85.610962
1,Aircraft observed a UAS above while W bound at...,DFW,DFW,RWY 18R,[5 NM],18R,-97.054815,32.893723,3.06900,183.06900,9.260,32.810345,-97.060108
2,Aircraft observed a UAS at the same altitude w...,MEM,MEM,RWY 36L,[6 NM],36L,-89.987278,35.039251,-1.62738,358.37262,11.112,35.139371,-89.990741
3,Aircraft observed a red UAS off the right side...,SEA,SEA,RWY 16R,[4 NM],16R,-122.317932,47.449860,15.40471,175.40471,7.408,47.383443,-122.310072
4,Aircraft observed a small red UAS E bound off ...,ADS,ADS,RWY 15,[3 NM],15,-96.837304,32.970408,2.94849,152.94849,5.556,32.925788,-96.810288
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Aircraft reported a UAS passing 100 feet below...,TLH,TLH,RWY 27,[2 NM],27,-84.346397,30.391355,-4.96630,265.03370,3.704,30.388457,-84.384793
91,"Aircraft reported a UAS off the right side, ap...",LAX,LAX,RWY 25L,[3 NM],25L,-118.404501,33.935085,11.71212,261.71212,5.556,33.927851,-118.463962
92,"Aircraft reported a grey, basketball size, UAS...",LAX,LAX,RWY 25L,[10 NM],25L,-118.404501,33.935085,11.71212,261.71212,18.520,33.910858,-118.602666
93,Aircraft reported a UAS off the left side whil...,RDU,RDU,RWY 23R,[3 NM],23R,-78.792358,35.882177,-9.22834,220.77166,5.556,35.844248,-78.832521
